**Regular CNN**

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import numpy as np
from PIL import Image
from tensorflow.keras.applications import VGG16

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

# Normalize pixel values to be between 0 and 1
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Convert labels to one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

In [ ]:
model = keras.Sequential([
    layers.Conv2D(filters=32, kernel_size=(5,5), padding='same', activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPool2D(strides=2),
    layers.Conv2D(filters=48, kernel_size=(5,5), padding='valid', activation='relu'),
    #layers.Conv2D(filters=56, kernel_size=(6,6), padding='valid', activation='relu'),
    layers.MaxPool2D(strides=2),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(84, activation='relu'),
    layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# Train the model
history = model.fit(x_train, y_train, epochs=20, batch_size=512, validation_data=(x_test, y_test))

Epoch 1/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 97s 795ms/step - accuracy: 0.6103 - loss: 1.1250 - val_accuracy: 0.8293 - val_loss: 0.4742
Epoch 2/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 88s 744ms/step - accuracy: 0.8337 - loss: 0.4563 - val_accuracy: 0.8641 - val_loss: 0.3732
Epoch 3/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 89s 755ms/step - accuracy: 0.8675 - loss: 0.3694 - val_accuracy: 0.8781 - val_loss: 0.3370
Epoch 4/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 141s 750ms/step - accuracy: 0.8789 - loss: 0.3266 - val_accuracy: 0.8858 - val_loss: 0.3149
Epoch 5/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 141s 745ms/step - accuracy: 0.8906 - loss: 0.3016 - val_accuracy: 0.8863 - val_loss: 0.3078
Epoch 6/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 87s 741ms/step - accuracy: 0.8985 - loss: 0.2755 - val_accuracy: 0.8911 - val_loss: 0.2986
Epoch 7/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 91s 771ms/step - accuracy: 0.9034 - loss: 0.2646 - val_accuracy: 0.8993 - val_loss: 0.2733
Epoch 8/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 89s 757ms/step - accuracy: 0.9098 - loss:

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

Test loss: 0.2351500540971756
Test accuracy: 0.9175000190734863


По сравнению с заданием 9, в котором результат был Test accuracy: 0.9030, CNN дало лучше результат (была сложность с переобучением). Также CNN требует ьольше времени.

**VGG16**

Выделение признаков

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
# Change image size

X_train_resized = []
X_test_resized = []

for image in x_train:
    image = Image.fromarray(image)
    image = image.resize((32, 32))
    image = np.array(image)
    X_train_resized.append(image)

for image in x_test:
    image = Image.fromarray(image)
    image = image.resize((32, 32))
    image = np.array(image)
    X_test_resized.append(image)

X_train_resized = np.array(X_train_resized)
X_test_resized = np.array(X_test_resized)

In [4]:
# Add a third channel to the images

X_train_resized = np.repeat(X_train_resized[..., np.newaxis], 3, -1)
X_test_resized = np.repeat(X_test_resized[..., np.newaxis], 3, -1)

In [5]:
# Preprocess the images

X_train_prepared = tf.keras.applications.vgg16.preprocess_input(X_train_resized)
X_test_prepared = tf.keras.applications.vgg16.preprocess_input(X_test_resized)

In [6]:
# one-hot encoding
Y_train_prepared = tf.keras.utils.to_categorical(y_train, num_classes=10)
Y_test_prepared = tf.keras.utils.to_categorical(y_test, num_classes=10)

In [7]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
base_model.trainable = False

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [59]:
from keras import regularizers

model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
# model.add(layers.Dense(64, activation='relu', kernel_initializer="he_normal", kernel_regularizer=regularizers.l2())),
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))

In [60]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [61]:
model.fit(X_train_prepared, Y_train_prepared, epochs=50, batch_size=512, validation_split=0.2)

Epoch 1/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 11s 96ms/step - accuracy: 0.5267 - loss: 8.4480 - val_accuracy: 0.7830 - val_loss: 0.7217
Epoch 2/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 5s 55ms/step - accuracy: 0.7322 - loss: 0.9185 - val_accuracy: 0.8058 - val_loss: 0.5846
Epoch 3/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 11s 58ms/step - accuracy: 0.7635 - loss: 0.6978 - val_accuracy: 0.8156 - val_loss: 0.5359
Epoch 4/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 10s 59ms/step - accuracy: 0.7902 - loss: 0.6076 - val_accuracy: 0.8245 - val_loss: 0.5026
Epoch 5/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 10s 56ms/step - accuracy: 0.8029 - loss: 0.5557 - val_accuracy: 0.8313 - val_loss: 0.4825
Epoch 6/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 5s 57ms/step - accuracy: 0.8154 - loss: 0.5143 - val_accuracy: 0.8332 - val_loss: 0.4678
Epoch 7/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 10s 59ms/step - accuracy: 0.8209 - loss: 0.4868 - val_accuracy: 0.8423 - val_loss: 0.4598
Epoch 8/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 10s 56ms/step - accuracy: 0.8312 - loss: 0.4539 - val_accuracy: 0.84

In [62]:
test_loss, test_accuracy = model.evaluate(X_test_prepared, Y_test_prepared)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.8671 - loss: 0.4822
Test Loss: 0.4937053620815277
Test Accuracy: 0.8672000169754028


Максимум был результат 0,87, лучше не нашел(либо с переобучением или высоким val_loss)

Дообучение

In [64]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

In [65]:
# Change image size

X_train_resized = []
X_test_resized = []

for image in x_train:
    image = Image.fromarray(image)
    image = image.resize((32, 32))
    image = np.array(image)
    X_train_resized.append(image)

for image in x_test:
    image = Image.fromarray(image)
    image = image.resize((32, 32))
    image = np.array(image)
    X_test_resized.append(image)

X_train_resized = np.array(X_train_resized)
X_test_resized = np.array(X_test_resized)

In [66]:
# Add a third channel to the images

X_train_resized = np.repeat(X_train_resized[..., np.newaxis], 3, -1)
X_test_resized = np.repeat(X_test_resized[..., np.newaxis], 3, -1)

In [67]:
# Preprocess the images

X_train_prepared = tf.keras.applications.vgg16.preprocess_input(X_train_resized)
X_test_prepared = tf.keras.applications.vgg16.preprocess_input(X_test_resized)

In [68]:
# one-hot encoding
Y_train_prepared = tf.keras.utils.to_categorical(y_train, num_classes=10)
Y_test_prepared = tf.keras.utils.to_categorical(y_test, num_classes=10)

In [69]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
base_model.trainable = False

In [70]:
# Разморозка некоторых шаров

for layer in base_model.layers[-5:]:
    layer.trainable = True

In [71]:
from keras import regularizers

model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
# model.add(layers.Dense(64, activation='relu', kernel_initializer="he_normal", kernel_regularizer=regularizers.l2())),
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))

In [72]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [73]:
model.fit(X_train_prepared, Y_train_prepared, epochs=50, batch_size=512, validation_split=0.2)

Epoch 1/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 16s 115ms/step - accuracy: 0.1048 - loss: 7.7737 - val_accuracy: 0.1030 - val_loss: 2.3026
Epoch 2/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 7s 70ms/step - accuracy: 0.0974 - loss: 2.3027 - val_accuracy: 0.0983 - val_loss: 2.3026
Epoch 3/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 7s 72ms/step - accuracy: 0.1002 - loss: 2.3026 - val_accuracy: 0.0957 - val_loss: 2.3027
Epoch 4/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 10s 69ms/step - accuracy: 0.1023 - loss: 2.3026 - val_accuracy: 0.0957 - val_loss: 2.3027
Epoch 5/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 7s 71ms/step - accuracy: 0.0992 - loss: 2.3026 - val_accuracy: 0.0957 - val_loss: 2.3027
Epoch 6/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 10s 72ms/step - accuracy: 0.0996 - loss: 2.3026 - val_accuracy: 0.0957 - val_loss: 2.3027
Epoch 7/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 7s 73ms/step - accuracy: 0.1013 - loss: 2.3026 - val_accuracy: 0.0957 - val_loss: 2.3027
Epoch 8/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 10s 72ms/step - accuracy: 0.1015 - loss: 2.3025 - val_accuracy: 0.095

In [74]:
test_loss, test_accuracy = model.evaluate(X_test_prepared, Y_test_prepared)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.0978 - loss: 2.3027
Test Loss: 2.302597761154175
Test Accuracy: 0.10000000149011612


В данном случае дообучение полностью провалилось Test Accuracy: 0.10000000149011612